In [16]:
import imageio
import numpy as np
from pathlib import Path
from omegaconf import OmegaConf
import os
from IPython.display import display, Image, HTML
os.environ["OPENCV_IO_ENABLE_OPENEXR"] = "1"
import cv2 as cv
import matplotlib.pyplot as plt
from sklearn import linear_model
import torch.nn.functional as F
import torch
import yaml
import pandas as pd

def disp_im(im):
    plt.close()
    plt.cla()
    fig = plt.imshow(im)
    plt.close()
    display(fig.figure)


In [31]:
exps = ["norussian", "noprednorms_nl0", "noprednorms_nl0_conserve_pb0", 'fixedmip128']
# log_dir = Path("../log") / "norussian"
# log_dir = Path("../log") / "noprednorms_nl0"
rows = []
col_keys = ['psnr', 'ssim', 'l_alex', 'envmap_psnr', 'envmap_smape']
col_rows = {k: [] for k in col_keys}
for exp in exps:
    log_dir = Path("../log") / exp
    col_data = {k: dict(exp=exp) for k in col_keys}
    for p in log_dir.glob("*"):
        pano_path = p / "imgs_test_all" / "envmaps" / "pano.exr"
        mapped_pano_path = p / "imgs_test_all" / "envmaps" / "mapped_pano.png"
        old_data_path = p / "imgs_test_all" / "stats.yaml"
        new_data_path = p / "imgs_test_all" / "stats_aug.yaml"
        config = OmegaConf.load(p / 'config.yaml')
        dname = Path(config['dataset']['scenedir']).name
        if new_data_path.exists():
            with new_data_path.open('r') as f:
                data = yaml.safe_load(f)
        else:
            with old_data_path.open('r') as f:
                data = yaml.safe_load(f)

        if dname == 'shinyball':
            continue
        data['dname'] = dname
        data['exp'] = exp
        rows.append(data)
        for k in col_keys:
            if k in data:
                col_data[k][dname] = data[k]
    for k in col_keys:
        col_data[k]['mean'] = np.mean([v for k, v in col_data[k].items() if type(v) != str])
        col_rows[k].append(col_data[k])

df = pd.DataFrame.from_records(rows)
psnrs = pd.DataFrame.from_dict(col_rows['psnr'])
ssims = pd.DataFrame.from_dict(col_rows['ssim'])
lpips = pd.DataFrame.from_dict(col_rows['l_alex'])
epsnrs = pd.DataFrame.from_dict(col_rows['envmap_psnr'])
display(HTML(psnrs.to_html()))
display(HTML(ssims.to_html()))
display(HTML(lpips.to_html()))
display(HTML(epsnrs.to_html()))

/home/dronelab/miniconda3/envs/31/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/dronelab/miniconda3/envs/31/lib/python3.10/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


,exp,car,ficus,ball,toaster,ship,coffee,lego,teapot,helmet,hotdog,drums,mic,materials,chair,mean
0,norussian,30.988888,29.906166,36.467042,26.964863,26.374857,30.042825,33.497285,41.695678,32.984400,33.029449,24.490492,32.852200,30.946286,32.240708,31.605796
1,noprednorms_nl0,NaN,29.992312,36.480335,26.897224,26.946890,29.952949,33.607760,41.010536,33.612827,33.454925,24.921478,33.011412,31.074967,32.840426,31.831080
2,noprednorms_nl0_conserve_pb0,30.517246,NaN,36.680686,NaN,NaN,NaN,NaN,41.176898,34.101595,NaN,NaN,32.932520,30.934505,NaN,34.390575
3,fixedmip128,NaN,29.365387,31.853813,26.684763,26.916537,28.286087,33.236467,NaN,34.626613,33.544547,24.337377,30.590943,NaN,32.231430,30.152178


,exp,car,ficus,ball,toaster,ship,coffee,lego,teapot,helmet,hotdog,drums,mic,materials,chair,mean
0,norussian,0.957110,0.944262,0.973434,0.884439,0.805577,0.925479,0.956506,0.981527,0.955253,0.937132,0.807241,0.954970,0.954553,0.947110,0.927471
1,noprednorms_nl0,NaN,0.921249,0.930853,0.856212,0.779609,0.863214,0.906821,0.882615,0.868878,0.909750,0.791618,0.939790,0.956805,0.903082,0.885423
2,noprednorms_nl0_conserve_pb0,0.926505,NaN,0.944931,NaN,NaN,NaN,NaN,0.910647,0.874415,NaN,NaN,0.944883,0.954730,NaN,0.926018
3,fixedmip128,NaN,0.933087,0.948631,0.878096,0.812080,0.921281,0.955889,NaN,0.966024,0.940197,0.818707,0.938885,NaN,0.950123,0.914818


,exp,car,ficus,ball,toaster,ship,coffee,lego,teapot,helmet,hotdog,drums,mic,materials,chair,mean
0,norussian,0.070179,0.095747,0.110445,0.195631,0.189808,0.174897,0.069957,0.098445,0.134390,0.133426,0.288888,0.112292,0.045602,0.119780,0.131392
1,noprednorms_nl0,NaN,0.190231,0.207120,0.241559,0.228518,0.305532,0.177081,0.366465,0.302283,0.197278,0.330854,0.166874,0.041841,0.266821,0.232497
2,noprednorms_nl0_conserve_pb0,0.185081,NaN,0.184181,NaN,NaN,NaN,NaN,0.315498,0.282733,NaN,NaN,0.152912,0.052395,NaN,0.195467
3,fixedmip128,NaN,0.147854,0.141328,0.197954,0.191348,0.182495,0.065763,NaN,0.115109,0.130365,0.284076,0.129661,NaN,0.102538,0.153499


,exp,car,ficus,toaster,ship,teapot,helmet,hotdog,drums,mic,materials,mean,ball
0,norussian,12.193773,13.147721,15.723842,13.897788,10.807273,17.918806,13.461659,14.371591,14.267512,17.085077,14.287504,NaN
1,noprednorms_nl0,NaN,15.939492,16.099970,14.804687,13.919299,18.492239,15.022659,15.311623,14.030765,15.885793,15.500725,NaN
2,noprednorms_nl0_conserve_pb0,13.246862,NaN,NaN,NaN,14.372873,18.492431,NaN,NaN,14.015710,15.825535,15.078050,14.514889
3,fixedmip128,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
